# CosmosDB

In [36]:
import os
import pymongo
from pymongo import UpdateOne, DeleteMany
from pprint import pprint

from bson.objectid import ObjectId

from dotenv import load_dotenv
load_dotenv()

from models import (

    Department,    # This we saw in previous lab

    Course,        # This represents a course offered in Fall 24 / Spring 25, 
                   # it has a section field to hold Section objects

    CalendarCourse,# This is a general course from the calendar, does not have sections, embeddings for courses are stored here

    Degree         # This is from the calendar too, represents a university program

)

### Connect to CosmosDB instance

In [6]:
# Get the CosmosDB connection string from .env file
CONNECTION_STRING = os.getenv("DB_CONNECTION_STRING")

# Establish connection to CosmosDB
db_client = pymongo.MongoClient(CONNECTION_STRING)

# Store database into a variable
db = db_client.db

# Store collections into variables
departments = db.departments
courses = db.courses
calendar_courses = db.calendar_courses
degrees = db.degrees

/var/folders/b6/zjrzfvq14h12kvt3_h7c4fwr0000gn/T/ipykernel_20200/4107983484.py:5: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  db_client = pymongo.MongoClient(CONNECTION_STRING)


### Insert into MongoDB

MongoDB collections hold JSON. 

One method of creating a document is using the `insert_one` method.

In [8]:
dept = Department(
        code="ECE",    
        pid="ryAe4JY7V", 
        name="Electrical and Computer Engineering", 
        faculty="Engineering and Computer Science")

# Generate JSON
dept_json = dept.model_dump()

# Insert the JSON into the database, and retrieve the inserted/generated ID
departments.insert_one(dept_json).inserted_id

ObjectId('66723bf8370e25ade8e3882d')

Since our JSON that we inserted did not have an **_id** field, the ID ObjectId('66723bf8370e25ade8e3882d') was auto generated.

If we had _id on the json it would have used that instead.

Another reason to use pydantic model is that it allows specifying aliases on object attributes and can be converted to JSON with alias names like so 

```python
dept.model_dump(by_alias=True)
```

### Retrieve a document

The `find_one` method is used to retrieve a single document from the database.

In [20]:
doc = departments.find_one({'_id': ObjectId('66723bf8370e25ade8e3882d')})

doc

{'_id': ObjectId('66723bf8370e25ade8e3882d'),
 'code': 'ECE',
 'pid': 'ryAe4JY7V',
 'name': 'Electrical and Computer Engineering',
 'faculty': 'Engineering and Computer Science'}

In [22]:
# Cast JSON document into the Product model
dept = Department(**doc)

dept

Department(code='ECE', pid='ryAe4JY7V', name='Electrical and Computer Engineering', faculty='Engineering and Computer Science')

**Note:** Since when we put the data into the database it came from a Department model, there should never be any issue in casting to Product model. 

So long as aliases are setup properly...

### Update a document

`find_one_and_update` method is used to update a single document in the database. 

Find additional examples of queries in the [documentation](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/tutorial-query).

In [29]:
result = departments.find_one_and_update(
    {"_id": ObjectId('66723bf8370e25ade8e3882d')},
    {"$set": {"code": "CSC"}},
    return_document=pymongo.ReturnDocument.AFTER)

type(result), result

(dict,
 {'_id': ObjectId('66723bf8370e25ade8e3882d'),
  'code': 'CSC',
  'pid': 'ryAe4JY7V',
  'name': 'Electrical and Computer Engineering',
  'faculty': 'Engineering and Computer Science'})

In [30]:
# Cast into Department object

type(Department(**result)), Department(**result)

(models.department.Department,
 Department(code='CSC', pid='ryAe4JY7V', name='Electrical and Computer Engineering', faculty='Engineering and Computer Science'))

### Delete a document

`delete_one` method is used to delete a single document from the database.

In [33]:
result = departments.delete_one({"_id": ObjectId('66723bf8370e25ade8e3882d')})

result

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [35]:
departments.count_documents({})

0

### Bulk writing

Bulk writing is important because much of the cost in insert into database is just establishing connection, not actually storing the data.

By bulk writing you only have to connect once. As opposed to individual writes, which can make queries take a very long time for no good reason...

In [ ]:
# PSEUDO-CODE: Don't try to actually execute this cell. 
# 
#              Just get the idea

# Get list of JSON objects. Typically this is done by loading from a .json file
dept_json_list = [...]

# Create list of BaseModel objects that you want to insert into database (not strictly nececssarry, but good practice)
dept_list = [Department(**x) for x in dept_json_list]

# Create a list of bulk operations. Here we are only inserting, but that can be done with UpdateOne and specifying upsert=True
#   result of this is a list of UpdateOne objects
bulk_operations = [UpdateOne({"_id": prod.id}, {"$set": prod.model_dump(by_alias=True)}, upsert=True) 
                   for dept in dept_list]

# Send the operations to db with bulk_write on the collection
departments.bulk_write(bulk_operations)

In [ ]:
# Now to find we can get all collections with a specified faculty (or whatever) value
departments.find({"faculty": "Engineering and Computer Science"}):

In [ ]:
# Alternatively we could match against a regex
for result in collection.find({"faculty": {"$regex": "bananas"}}):
    pprint(result)

**Note:** Reading the mongo db queries takes some time. But it starts to make sense after a bit.

I think it's particularly difficult with pymongo, cause everything needs to be wrapped in a string. 

The key is that it is uses nested dictionaries.

### Delete entire database

The following cell will delete the database and collections using the `drop_database` method on the database object.

To drop just a collection use `drop_collection`

In [ ]:
# db.drop_collection("departments")
client.drop_database("db")
client.close()